In [36]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory

import warnings
warnings.filterwarnings("ignore")

In [37]:
# Load dataset from directory
train_dir = 'datasets/train'
test_dir = 'datasets/test'

# Load train and test datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int', 
    shuffle=True,
    seed=123 
)

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int',
    shuffle=False, 
    seed=123
)

Found 1034 files belonging to 3 classes.
Found 128 files belonging to 3 classes.


In [38]:
# Function to extract features from the images using CNN
def extract_features(dataset, cnn_model):
    features = []
    labels = []

    for images, lbls in dataset:
        try:
            feature_batch = cnn_model.predict(images)
            features.append(feature_batch)
            labels.append(lbls.numpy()) # Convert labels to numpy arrays
        except Exception as e:
            print("Error processing a batch:", e)
            continue

    features = np.vstack(features) # Shape to (samples, features)
    labels = np.concatenate(labels) # Combine all label batches
    return features, labels

In [39]:
# Load pre-trained CNN model (VGG16 without top layer)
cnn_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [40]:
# Extract features from train and test sets
train_features, train_labels = extract_features(train_dataset, cnn_model)
test_features, test_labels = extract_features(test_dataset, cnn_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

2024-09-23 18:39:22.722851: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [42]:
# Flatten the features
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

In [43]:
# Normalize the features for SVM
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

In [47]:
# SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(train_features_scaled, train_labels)

# Predict and evaluate
test_predictions = svm_model.predict(test_features_scaled)
train_preditions = svm_model.predict(train_features_scaled)
test_accuracy = accuracy_score(test_labels, test_predictions)
train_accuracy = accuracy_score(train_labels, train_preditions)
print(f"Test Accuracy: {test_accuracy:.3f}")
print("Train accuracy: {:.3f}".format(train_accuracy))

Test Accuracy: 0.898
Train accuracy: 1.000


In [50]:
import pickle

with open("models/vgg16_svm_ensemble.pkl", "wb") as file:
    pickle.dump(svm_model, file)